
<img src="all teams.jpg" style ="width:800px"/>
<img src="nbalogo.jpg" style ="width:200px"/>
<h1>ESPN NBA Web Scraping Roster & Player Stats</h1>

In [91]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep
import urllib.request
import json
from itertools import chain


In [2]:
#Team names
request = requests.get('http://www.espn.com/nba/teams')
soup = BeautifulSoup(request.content, 'html.parser')
teamNames = []
rosterUrl = []
names = soup.find_all('h2', {'class': 'di clr-gray-01 h5'})
for n in names:
    teamNames.append(n.get_text())
#print(teamNames)
#https://www.espn.com/nba/team/roster/_/name/atl/atlanta-hawks

url = soup.find_all('div', {'class' : 'TeamLinks__Links'})
for u in url:
    b = u.find_all('span', {'class' : 'TeamLinks__Link n9 nowrap'})
    z = b[2].find_all(href=True)
    for u in z:
        rosterUrl.append(u['href'])


In [3]:
teamNames

['Boston Celtics',
 'Brooklyn Nets',
 'New York Knicks',
 'Philadelphia 76ers',
 'Toronto Raptors',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Detroit Pistons',
 'Indiana Pacers',
 'Milwaukee Bucks',
 'Denver Nuggets',
 'Minnesota Timberwolves',
 'Oklahoma City Thunder',
 'Portland Trail Blazers',
 'Utah Jazz',
 'Golden State Warriors',
 'LA Clippers',
 'Los Angeles Lakers',
 'Phoenix Suns',
 'Sacramento Kings',
 'Atlanta Hawks',
 'Charlotte Hornets',
 'Miami Heat',
 'Orlando Magic',
 'Washington Wizards',
 'Dallas Mavericks',
 'Houston Rockets',
 'Memphis Grizzlies',
 'New Orleans Pelicans',
 'San Antonio Spurs']

In [4]:
rosterUrl

['/nba/team/roster/_/name/bos/boston-celtics',
 '/nba/team/roster/_/name/bkn/brooklyn-nets',
 '/nba/team/roster/_/name/ny/new-york-knicks',
 '/nba/team/roster/_/name/phi/philadelphia-76ers',
 '/nba/team/roster/_/name/tor/toronto-raptors',
 '/nba/team/roster/_/name/chi/chicago-bulls',
 '/nba/team/roster/_/name/cle/cleveland-cavaliers',
 '/nba/team/roster/_/name/det/detroit-pistons',
 '/nba/team/roster/_/name/ind/indiana-pacers',
 '/nba/team/roster/_/name/mil/milwaukee-bucks',
 '/nba/team/roster/_/name/den/denver-nuggets',
 '/nba/team/roster/_/name/min/minnesota-timberwolves',
 '/nba/team/roster/_/name/okc/oklahoma-city-thunder',
 '/nba/team/roster/_/name/por/portland-trail-blazers',
 '/nba/team/roster/_/name/utah/utah-jazz',
 '/nba/team/roster/_/name/gs/golden-state-warriors',
 '/nba/team/roster/_/name/lac/la-clippers',
 '/nba/team/roster/_/name/lal/los-angeles-lakers',
 '/nba/team/roster/_/name/phx/phoenix-suns',
 '/nba/team/roster/_/name/sac/sacramento-kings',
 '/nba/team/roster/_/nam

In [5]:
roster_data = dict(zip(teamNames,rosterUrl))
roster_data

{'Boston Celtics': '/nba/team/roster/_/name/bos/boston-celtics',
 'Brooklyn Nets': '/nba/team/roster/_/name/bkn/brooklyn-nets',
 'New York Knicks': '/nba/team/roster/_/name/ny/new-york-knicks',
 'Philadelphia 76ers': '/nba/team/roster/_/name/phi/philadelphia-76ers',
 'Toronto Raptors': '/nba/team/roster/_/name/tor/toronto-raptors',
 'Chicago Bulls': '/nba/team/roster/_/name/chi/chicago-bulls',
 'Cleveland Cavaliers': '/nba/team/roster/_/name/cle/cleveland-cavaliers',
 'Detroit Pistons': '/nba/team/roster/_/name/det/detroit-pistons',
 'Indiana Pacers': '/nba/team/roster/_/name/ind/indiana-pacers',
 'Milwaukee Bucks': '/nba/team/roster/_/name/mil/milwaukee-bucks',
 'Denver Nuggets': '/nba/team/roster/_/name/den/denver-nuggets',
 'Minnesota Timberwolves': '/nba/team/roster/_/name/min/minnesota-timberwolves',
 'Oklahoma City Thunder': '/nba/team/roster/_/name/okc/oklahoma-city-thunder',
 'Portland Trail Blazers': '/nba/team/roster/_/name/por/portland-trail-blazers',
 'Utah Jazz': '/nba/tea

In [6]:
#format {"name":"Trae Young","href":"https://www.espn.com/nba/player/_/id/4277905/"}
url = "https://www.espn.com/nba/team/roster/_/name/gs/golden-state-warriors"
request = urllib.request.urlopen(url)
src = request.read().decode('utf-8')
#on each roster page we can return player information
player_regex = ('\{\"name\"\:\"(\w+\s\w+)\",\"href\"\:\"https\://www\.espn\.com/nba/player/.*?\",(.*?)\}')

info = re.findall(player_regex, src)
info #List of tuples with player information

[('Kent Bazemore',
  '"uid":"s:40~l:46~a:6637","guid":"1b9fcc3b945fa8c17aa9a72b5dab4fa5","id":"6637","height":"6\' 5\\"","weight":"206 lbs","age":31,"position":"SF","jersey":"26","salary":"$1,620,564","birthDate":"07/01/89","headshot":"https://a.espncdn.com/i/headshots/nba/players/full/6637.png","lastName":"Kent Bazemore","experience":8,"college":"Old Dominion","birthPlace":"Kelford, NC"'),
 ('Marquese Chriss',
  '"uid":"s:40~l:46~a:3907487","guid":"a320ecf1d6481b7518ddc1dc576c27b4","id":"3907487","height":"6\' 9\\"","weight":"249 lbs","age":23,"position":"SF","jersey":"32","salary":"$1,824,003","birthDate":"07/02/97","headshot":"https://a.espncdn.com/i/headshots/nba/players/full/3907487.png","lastName":"Marquese Chriss","experience":4,"college":"Washington","birthPlace":"SACRAMENTO, CA"'),
 ('Stephen Curry',
  '"uid":"s:40~l:46~a:3975","guid":"5dda51f150c966e12026400b73f34fad","id":"3975","height":"6\' 3\\"","weight":"193 lbs","age":32,"position":"PG","jersey":"30","salary":"$43,006,3

In [7]:
steph_curry = json.loads("{"+ info[2][1]+"}")
steph_curry

{'uid': 's:40~l:46~a:3975',
 'guid': '5dda51f150c966e12026400b73f34fad',
 'id': '3975',
 'height': '6\' 3"',
 'weight': '193 lbs',
 'age': 32,
 'position': 'PG',
 'jersey': '30',
 'salary': '$43,006,362',
 'birthDate': '03/14/88',
 'headshot': 'https://a.espncdn.com/i/headshots/nba/players/full/3975.png',
 'lastName': 'Stephen Curry',
 'experience': 11,
 'college': 'Davidson',
 'birthPlace': 'Akron, OH'}

In [44]:
#create a dictionary for each team and get player info
def get_nba_player_info(roster_url):
    request = urllib.request.urlopen(roster_url)
    src = request.read().decode('utf-8')
    #on each roster page we can return player information
    player_regex = ('\{\"name\"\:\"(\w+\s\w+)\",\"href\"\:\"https\://www\.espn\.com/nba/player/.*?\",(.*?)\}')
    info = re.findall(player_regex, src)
    player_dict = {}
    for player in info:
        player_dict[player[0]] = json.loads("{"+ player[1]+"}") # assign value to key. Key being the player's name and value is the player information
    
    return player_dict



In [9]:
#nested dictionaries-> 
#team => all players
    #player name => player details
        #player details dictionary id, height, weight etc
all_nba_players={}
for team in roster_data.keys():
    all_nba_players[team] = get_nba_player_info("https://www.espn.com"+roster_data[team])  


In [10]:
all_nba_players.keys()

dict_keys(['Boston Celtics', 'Brooklyn Nets', 'New York Knicks', 'Philadelphia 76ers', 'Toronto Raptors', 'Chicago Bulls', 'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Milwaukee Bucks', 'Denver Nuggets', 'Minnesota Timberwolves', 'Oklahoma City Thunder', 'Portland Trail Blazers', 'Utah Jazz', 'Golden State Warriors', 'LA Clippers', 'Los Angeles Lakers', 'Phoenix Suns', 'Sacramento Kings', 'Atlanta Hawks', 'Charlotte Hornets', 'Miami Heat', 'Orlando Magic', 'Washington Wizards', 'Dallas Mavericks', 'Houston Rockets', 'Memphis Grizzlies', 'New Orleans Pelicans', 'San Antonio Spurs'])

In [11]:
all_nba_players["Toronto Raptors"]["Pascal Siakam"]["salary"]
# ummmm he isn't playing like an all-star

'$30,559,200'

In [12]:
all_nba_players["Toronto Raptors"]["Malachi Flynn"]["salary"]
#give him minutes!!!

'$1,950,600'

In [13]:
all_nba_players["Charlotte Hornets"]["LaMelo Ball"]["salary"]

'$7,839,960'

In [14]:
all_nba_players["Los Angeles Lakers"]["LeBron James"]["salary"]

'$39,219,565'

In [15]:
all_nba_players["Los Angeles Lakers"]["Anthony Davis"]["salary"]

'$32,742,000'

In [16]:
#Export each team roster information as a dataframe and then to a CSV
toronto_raptors_pd = pd.DataFrame.from_dict(all_nba_players['Toronto Raptors'], orient= 'index')

In [17]:
toronto_raptors_pd

,uid,guid,id,height,weight,age,position,hand,jersey,salary,birthDate,headshot,lastName,experience,college,birthPlace
OG Anunoby,s:40~l:46~a:3934719,31ec117d6e14d4f094149d2467197179,3934719,"6' 8""",232 lbs,23,SF,B,3,"$3,872,215",07/17/97,https://a.espncdn.com/i/headshots/nba/players/...,OG Anunoby,3,Indiana,England
Aron Baynes,s:40~l:46~a:2968439,e13c65978f551b2168dd677c0182c52d,2968439,"6' 10""",260 lbs,34,C,NaN,46,"$7,000,000",12/09/86,https://a.espncdn.com/i/headshots/nba/players/...,Aron Baynes,8,Washington State,New Zealand
Chris Boucher,s:40~l:46~a:3948153,843d6489a236c54a38db617c74ded459,3948153,"6' 10""",200 lbs,27,PF,NaN,25,"$6,750,000",01/11/93,https://a.espncdn.com/i/headshots/nba/players/...,Chris Boucher,3,Oregon,NaN
Terence Davis,s:40~l:46~a:3919454,063803e61039bb717172aa9fe499dbe5,3919454,"6' 4""",205 lbs,23,SG,NaN,0,"$1,517,981",05/16/97,https://a.espncdn.com/i/headshots/nba/players/...,Terence Davis,1,Ole Miss,"Southaven, MS"
Malachi Flynn,s:40~l:46~a:4066668,6feae4fc4a16e3800f60b635cd76ebdf,4066668,"6' 1""",185 lbs,22,PG,NaN,8,"$1,950,600",05/10/98,https://a.espncdn.com/i/headshots/nba/players/...,Malachi Flynn,0,San Diego State,"Tacoma, WA"
Jalen Harris,s:40~l:46~a:4066998,7aa971e56695be8cc50fc9b5efb93d37,4066998,"6' 5""",195 lbs,22,SG,R,2,0,08/14/98,https://a.espncdn.com/i/headshots/nba/players/...,Jalen Harris,0,Nevada,"Dallas, TX"
Stanley Johnson,s:40~l:46~a:3134881,6caed88faf780c9839373c45c93fd069,3134881,"6' 7""",245 lbs,24,SF,NaN,5,"$3,804,150",05/29/96,https://a.espncdn.com/i/headshots/nba/players/...,Stanley Johnson,5,Arizona,"Anaheim, CA"
Alex Len,s:40~l:46~a:2596107,56df8855ab0a659aeed2ccb77a2b77f7,2596107,"7' 0""",250 lbs,27,C,NaN,27,"$2,258,000",06/16/93,https://a.espncdn.com/i/headshots/nba/players/...,Alex Len,7,Maryland,Ukraine
Kyle Lowry,s:40~l:46~a:3012,f47675222a75d36e0214eeb7a489af0e,3012,"6' 1""",196 lbs,34,PG,NaN,7,"$30,500,000",03/25/86,https://a.espncdn.com/i/headshots/nba/players/...,Kyle Lowry,14,Villanova,"Philadelphia, PA"
Patrick McCaw,s:40~l:46~a:3137730,a8813780dda6d3d8164dd05be84d7a43,3137730,"6' 7""",185 lbs,25,SF,NaN,22,"$4,000,000",10/25/95,https://a.espncdn.com/i/headshots/nba/players/...,Patrick McCaw,4,UNLV,NaN


In [62]:
#create a df with all players
def join_df_all_players():
    all_players = pd.DataFrame()
    for team in all_nba_players.keys():
        all_df = pd.DataFrame.from_dict(all_nba_players[team], orient= 'index')
        all_df['Team'] = team
        all_df.rename(columns ={'lastName': 'FLName'} ,inplace = True)
        all_players = all_players.append(all_df) 
    return all_players

In [63]:
nba_df= join_df_all_players()
nba_df

,uid,guid,id,height,weight,age,position,jersey,salary,birthDate,headshot,FLName,experience,college,birthPlace,Team,hand
Jaylen Brown,s:40~l:46~a:3917376,0d5cde01f6d3225fdae544ef3304cda2,3917376,"6' 6""",223 lbs,24,SG,7,"$23,735,118",10/24/96,https://a.espncdn.com/i/headshots/nba/players/...,Jaylen Brown,4,California,"Marietta, GA",Boston Celtics,NaN
Carsen Edwards,s:40~l:46~a:4066407,4b8ebdfd01221567925035c1e0d0c337,4066407,"5' 11""",200 lbs,22,PG,4,"$1,517,981",03/12/98,https://a.espncdn.com/i/headshots/nba/players/...,Carsen Edwards,1,Purdue,"Houston, TX",Boston Celtics,NaN
Tacko Fall,s:40~l:46~a:3904625,cec7bd80fe2880fe8d9efe9a2294a563,3904625,"7' 5""",311 lbs,25,C,99,0,12/10/95,https://a.espncdn.com/i/headshots/nba/players/...,Tacko Fall,1,UCF,Senegal,Boston Celtics,NaN
Javonte Green,s:40~l:46~a:2596112,a4940ed033e0a114e8862f5a094aa3f8,2596112,"6' 4""",205 lbs,27,SG,43,"$1,517,981",07/23/93,https://a.espncdn.com/i/headshots/nba/players/...,Javonte Green,1,Radford,"Alberta, VA",Boston Celtics,NaN
Romeo Langford,s:40~l:46~a:4397008,75220698859cebd0a78dd7abedc92687,4397008,"6' 4""",216 lbs,21,SG,45,"$3,631,200",10/25/99,https://a.espncdn.com/i/headshots/nba/players/...,Romeo Langford,1,Indiana,"New Albany, IN",Boston Celtics,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jakob Poeltl,s:40~l:46~a:3134908,a5b22bdbe9e3f48272f906db2e339aff,3134908,"7' 0""",240 lbs,25,C,25,"$8,101,852",10/15/95,https://a.espncdn.com/i/headshots/nba/players/...,Jakob Poeltl,4,Utah,Austria,San Antonio Spurs,NaN
Luka Samanic,s:40~l:46~a:4423309,bc9995ea4b1a9782ef30b0a4d7299aad,4423309,"6' 11""",235 lbs,20,PF,19,"$2,892,000",01/09/00,https://a.espncdn.com/i/headshots/nba/players/...,Luka Samanic,1,NaN,Croatia,San Antonio Spurs,NaN
Devin Vassell,s:40~l:46~a:4395630,4a925ef3a49b2fc8b55eda12e9bcfdb6,4395630,"6' 7""",195 lbs,20,SG,24,"$4,033,440",08/23/00,https://a.espncdn.com/i/headshots/nba/players/...,Devin Vassell,0,Florida State,"Lawrenceville, GA",San Antonio Spurs,NaN
Quinndary Weatherspoon,s:40~l:46~a:3907358,f40458f631cee819bf79e83079dc3aac,3907358,"6' 4""",205 lbs,24,SG,15,0,09/10/96,https://a.espncdn.com/i/headshots/nba/players/...,Quinndary Weatherspoon,1,Mississippi State,"Canton, MS",San Antonio Spurs,NaN


In [64]:
#export as a csv
import datetime
now = datetime.datetime.now()
nba_df.to_csv("NBA_Roster_players_info_"+str(now.year)+".csv")

In [217]:
#Get specific player career statistics
#https://www.espn.com/nba/player/stats/_/id/1966/lebron-james
#["Career","","1271","1270","38.4","9.9-19.6","50.3","1.5-4.3","34.4","5.8-8.0","73.5","1.2","6.2","7.4","7.4","0.8","1.6","1.8","3.5","27.1"]}
url = "https://www.espn.com/nba/player/stats/_/id/1966/lebron-james"
req = urllib.request.urlopen(url)
src = req.read().decode('utf-8')
stats_regex = ('\[\"Career\",\"\",(.*?)\]\},\{\"ttl\"\:\"Regular Season Totals\"')
career_stats = re.findall(stats_regex, src)
print(career_stats)


['"1272","1271","38.3","9.9-19.6","50.3","1.5-4.3","34.4","5.8-8.0","73.4","1.2","6.2","7.4","7.4","0.8","1.6","1.8","3.5","27.0"']


In [127]:
#Regular season averages
#clean up player stats
career_info = career_stats[0].replace("\"", "").split(",")
career_info = list(chain.from_iterable([i.split("-") for i in career_info]))
career_info = list(map(float,career_info))
print(career_info)



[1271.0, 1270.0, 38.4, 9.9, 19.6, 50.3, 1.5, 4.3, 34.4, 5.8, 8.0, 73.5, 1.2, 6.2, 7.4, 7.4, 0.8, 1.6, 1.8, 3.5, 27.1]


In [249]:
#store each player career stats in a DF

career_player_stats = pd.DataFrame(columns = ["GP","GS","MIN","FGM", "FGA","FG%","3PTM","3PTA","3P%","FTM","FTA","FT%","OR","DR","REB","AST","BLK","STL","PF","TO","PTS"])

for players in nba_df.index:
    url = "https://www.espn.com/nba/player/stats/_/id/"+ str(nba_df.loc[players]["id"])
    req = urllib.request.urlopen(url)
    src = req.read().decode('utf-8')
    career_stats_regex = ('\[\"Career\",\"\",(.*?)\]\},\{\"ttl\"\:\"Regular Season Totals\"')
    stats_regex = ('\[\"Career\",\"\",(.*?)\]\},\{\"ttl\"\:\"Regular Season Totals\"')
    career_info  = re.findall(career_stats_regex, src)
    try:
    #print(career_info)
        career_info = career_info[0].replace("\"", "").split(",")
        career_info = list(chain.from_iterable([i.split("-") for i in career_info]))
        career_info = list(map(float,career_info))
        d = pd.Series(career_info, index = career_player_stats.columns, name=players)
        #print(d)
        career_player_stats = career_player_stats.append(pd.Series(career_info, index = career_player_stats.columns, name=players))
    except:
        #player is a rookie and has not played a game
        print('')

In [ ]:
#  GP:Games Played
# GS:Games Started
# MIN:Minutes Per Game
# FGM:Field Goals Made Per Game
# FGA:Field Goals Attempted Per Game
# FG%:Field Goal Percentage
# 3PTM:3-Point Field Goals Made Per Game
# 3PTA:3-Point Field Goals Attempted Per Game
# 3P%:3-Point Field Goal Percentage
# FTM:Free Throws Made Per Game
# FTA:Free Throws Attempted
# FT%:Free Throw Percentage
# OR:Offensive Rebounds Per Game
# DR:Defensive Rebounds Per Game
# REB:Rebounds Per Game
# AST:Assists Per Game
# BLK:Blocks Per Game
# STL:Steals Per Game
# PF:Fouls Per Game
# TO:Turnovers Per Game
# PTS:Points Per Game
# 

In [253]:
career_player_stats.to_csv("NBA_Roster_player_statistics_"+str(now.year)+".csv")

In [289]:
rebound_stats = career_player_stats.loc[career_player_stats["REB"] > 10.0]
rebound_stats

,GP,GS,MIN,FGM,FGA,FG%,3PTM,3PTA,3P%,FTM,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
DeAndre Jordan,882.0,720.0,27.7,3.8,5.7,67.0,0.0,0.0,9.1,1.9,...,47.4,3.2,7.6,10.8,0.9,1.6,0.6,2.6,1.4,9.4
Joel Embiid,214.0,214.0,30.5,8.0,16.6,48.1,1.1,3.6,32.0,6.8,...,79.3,2.4,9.1,11.5,3.1,1.8,0.8,3.4,3.5,24.0
Dwight Howard,1119.0,1046.0,33.4,6.0,10.3,58.7,0.0,0.1,14.5,4.6,...,56.5,3.4,8.8,12.3,1.4,1.9,0.9,3.2,2.8,16.7
Andre Drummond,605.0,554.0,31.0,6.1,11.3,54.1,0.0,0.2,14.0,2.2,...,46.2,4.8,9.1,13.8,1.3,1.6,1.4,3.2,2.0,14.5
Kevin Love,715.0,632.0,32.0,6.0,13.6,44.2,1.7,4.6,37.0,4.5,...,82.8,2.8,8.4,11.1,2.4,0.5,0.7,2.1,2.0,18.2
Rudy Gobert,480.0,388.0,29.5,4.3,6.8,64.0,0.0,0.0,0.0,3.1,...,63.0,3.2,7.7,11.0,1.4,2.2,0.7,2.6,1.6,11.7
Anthony Davis,534.0,529.0,34.5,8.9,17.3,51.5,0.5,1.5,32.1,5.7,...,80.2,2.6,7.8,10.4,2.3,2.4,1.4,2.4,1.9,24.0
Deandre Ayton,116.0,109.0,31.2,7.4,13.0,57.0,0.0,0.2,20.0,1.9,...,74.6,3.4,7.3,10.7,1.8,1.1,0.8,3.0,1.9,16.8
Hassan Whiteside,415.0,320.0,26.5,5.7,9.8,58.2,0.0,0.1,32.0,2.2,...,60.6,3.3,8.3,11.7,0.7,2.4,0.6,2.6,1.6,13.6
Nikola Vucevic,604.0,543.0,30.3,7.0,14.0,49.9,0.5,1.4,34.0,1.6,...,74.5,2.7,7.6,10.2,2.6,0.9,0.9,2.5,1.7,16.2


In [290]:
#Players averaging more than 25 points per games
points_leader = career_player_stats.loc[career_player_stats["PTS"] >25.0]
points_leader



,GP,GS,MIN,FGM,FGA,FG%,3PTM,3PTA,3P%,FTM,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
Kevin Durant,855.0,855.0,36.9,9.2,18.7,49.3,1.9,4.9,38.1,6.8,...,88.3,0.7,6.3,7.1,4.1,1.1,1.1,1.9,3.1,27.0
LeBron James,1272.0,1271.0,38.3,9.9,19.6,50.3,1.5,4.3,34.4,5.8,...,73.4,1.2,6.2,7.4,7.4,0.8,1.6,1.8,3.5,27.0
James Harden,836.0,623.0,34.3,7.4,16.7,44.4,2.8,7.7,36.4,7.6,...,85.8,0.8,4.6,5.3,6.3,0.5,1.6,2.6,3.7,25.2


In [298]:
#Steals Leader

steals_leader = career_player_stats.loc[career_player_stats["STL"]>1.0, ["PTS","STL", "GP"]].head(5)
steals_leader

,PTS,STL,GP
Payton Pritchard,6.6,1.6,7.0
Marcus Smart,9.9,1.6,408.0
Jayson Tatum,17.6,1.2,232.0
Jeff Teague,12.5,1.2,778.0
Kemba Walker,19.9,1.3,661.0


In [460]:
all_nba_players_stats = nba_df.join(career_player_stats)
all_nba_players_stats.drop(["uid", "guid","headshot","college", "birthPlace", "birthDate"], 1, inplace=True)

In [461]:
all_nba_players_stats.head(10)


,id,height,weight,age,position,jersey,salary,FLName,experience,Team,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
Jaylen Brown,3917376,"6' 6""",223 lbs,24,SG,7,"$23,735,118",Jaylen Brown,4,Boston Celtics,...,67.8,0.9,3.6,4.5,1.5,0.4,0.9,2.4,1.5,13.5
Carsen Edwards,4066407,"5' 11""",200 lbs,22,PG,4,"$1,517,981",Carsen Edwards,1,Boston Celtics,...,68.4,0.2,1.1,1.3,0.7,0.1,0.3,0.9,0.4,3.3
Tacko Fall,3904625,"7' 5""",311 lbs,25,C,99,0,Tacko Fall,1,Boston Celtics,...,60.0,0.3,1.8,2.0,0.1,0.8,0.1,0.9,0.4,3.1
Javonte Green,2596112,"6' 4""",205 lbs,27,SG,43,"$1,517,981",Javonte Green,1,Boston Celtics,...,66.7,0.5,1.4,2.0,0.5,0.2,0.5,0.9,0.5,3.4
Romeo Langford,4397008,"6' 4""",216 lbs,21,SG,45,"$3,631,200",Romeo Langford,1,Boston Celtics,...,72.0,0.5,0.8,1.3,0.4,0.3,0.3,1.0,0.3,2.5
Aaron Nesmith,4396909,"6' 6""",215 lbs,21,SF,24,"$3,458,400",Aaron Nesmith,0,Boston Celtics,...,0.0,0.5,1.0,1.5,0.5,0.5,0.0,1.5,1.0,4.0
Semi Ojeleye,3056602,"6' 6""",240 lbs,26,PF,37,"$1,752,950",Semi Ojeleye,3,Boston Celtics,...,68.1,0.4,1.5,2.0,0.4,0.1,0.3,1.1,0.3,3.2
Payton Pritchard,4066354,"6' 2""",206 lbs,22,PG,11,"$2,193,480",Payton Pritchard,0,Boston Celtics,...,66.7,0.0,2.0,2.0,2.4,0.3,1.6,1.9,1.9,6.6
Marcus Smart,2990992,"6' 3""",220 lbs,26,PG,36,"$13,446,428",Marcus Smart,6,Boston Celtics,...,77.4,0.9,2.7,3.5,4.1,0.4,1.6,2.6,1.7,9.9
Jayson Tatum,4065648,"6' 8""",210 lbs,22,PF,0,"$9,897,120",Jayson Tatum,3,Boston Celtics,...,82.8,0.8,5.2,6.0,2.3,0.8,1.2,2.1,1.8,17.6


In [462]:
all_nba_players_stats['salary']=[int(re.sub(r'[^\d.]+', '', s)) if isinstance(s, str) else s for s in all_nba_players_stats['salary'].values]
def convert_height(height):
    split_height = height.split(" ")
    feet = float(split_height[0].replace("\'",""))
    inches = float(split_height[1].replace("\"",""))
    return (feet*12 + inches)
all_nba_players_stats['height'] = [convert_height(x) for x in all_nba_players_stats['height']]
    

In [463]:
all_nba_players_stats

,id,height,weight,age,position,jersey,salary,FLName,experience,Team,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
Jaylen Brown,3917376,78.0,223 lbs,24,SG,7,23735118.0,Jaylen Brown,4,Boston Celtics,...,67.8,0.9,3.6,4.5,1.5,0.4,0.9,2.4,1.5,13.5
Carsen Edwards,4066407,71.0,200 lbs,22,PG,4,1517981.0,Carsen Edwards,1,Boston Celtics,...,68.4,0.2,1.1,1.3,0.7,0.1,0.3,0.9,0.4,3.3
Tacko Fall,3904625,89.0,311 lbs,25,C,99,0.0,Tacko Fall,1,Boston Celtics,...,60.0,0.3,1.8,2.0,0.1,0.8,0.1,0.9,0.4,3.1
Javonte Green,2596112,76.0,205 lbs,27,SG,43,1517981.0,Javonte Green,1,Boston Celtics,...,66.7,0.5,1.4,2.0,0.5,0.2,0.5,0.9,0.5,3.4
Romeo Langford,4397008,76.0,216 lbs,21,SG,45,3631200.0,Romeo Langford,1,Boston Celtics,...,72.0,0.5,0.8,1.3,0.4,0.3,0.3,1.0,0.3,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jakob Poeltl,3134908,84.0,240 lbs,25,C,25,8101852.0,Jakob Poeltl,4,San Antonio Spurs,...,52.6,2.0,2.9,4.9,1.0,1.0,0.4,2.1,0.8,5.5
Luka Samanic,4423309,83.0,235 lbs,20,PF,19,2892000.0,Luka Samanic,1,San Antonio Spurs,...,66.7,0.2,1.8,2.0,1.2,0.4,0.0,1.2,0.4,4.0
Devin Vassell,4395630,79.0,195 lbs,20,SG,24,4033440.0,Devin Vassell,0,San Antonio Spurs,...,0.0,0.2,2.8,3.0,1.0,0.4,1.0,1.0,0.6,3.4
Quinndary Weatherspoon,3907358,76.0,205 lbs,24,SG,15,0.0,Quinndary Weatherspoon,1,San Antonio Spurs,...,50.0,0.1,0.5,0.6,1.0,0.1,0.3,0.7,0.5,1.1
